# Conic Optimization [DRAFT]

## Quadratic  problem

In [184]:
import Pkg
Pkg.add("Pajarito")
Pkg.add("HiGHS")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
   Installed HiGHS_jll ─ v1.4.2+0
   Installed HiGHS ───── v1.4.3
    Updating `~/.julia/environments/v1.8/Project.toml`
  [87dc4568] + HiGHS v1.4.3
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [87dc4568] + HiGHS v1.4.3
  [8fd58aa0] + HiGHS_jll v1.4.2+0
Precompiling project...
  ✓ HiGHS_jll
  ✓ HiGHS
  2 dependencies successfully precompiled in 8 seconds. 235 already precompiled.


In [15]:
using LinearAlgebra
using JuMP
using Hypatia


#V = [
#    0.6290  0.07;
#    0.3592  0.0752;
#    0.3592  0.0752;
#]

#V = [
#    0.6290  0.07;
#    0.3592  0.0752;
#    0.3592  0.0752;
#]

V = [
    0.6290  0.3592  0.3592;
    0.07    0.0752  0.0752;
]

q = 2
p = 3
n = 70
n_max = 10000


# setup JuMP model
opt = Hypatia.Optimizer(verbose = false)
model = Model(() -> opt)

@variable(model, x[1:3] >= 0, Int)
@constraint(model, sum(x) == 70)

model = JuMP.Model()
JuMP.@variable(model, x[1:p])
JuMP.@constraint(model, sum(x) == n)
mid = n_max / 2
JuMP.@constraint(model, vcat(mid, x .- mid) in MOI.NormInfinityCone(p + 1))
JuMP.@variable(model, hypo)
JuMP.@objective(model, Max, hypo)

JuMP.@variable(model, L[i in 1:q, j in 1:i])
JuMP.@variable(model, W[1:p, 1:q])
VW = V * W
JuMP.@constraints(
    model,
    begin
        [i in 1:q, j in 1:i], VW[i, j] == L[i, j]
        [i in 1:q, j in (i + 1):q], VW[i, j] == 0
        vcat(hypo, [L[i, i] for i in 1:q]) in MOI.GeometricMeanCone(1 + q)
        [i in 1:p], vcat(sqrt(q) * x[i], W[i, :]) in JuMP.SecondOrderCone()
    end
)

# solve and query solution
optimize!(model)
termination_status(model)
objective_value(model)
value.(x)


LoadError: Constraints of type MathOptInterface.VariableIndex-in-MathOptInterface.Integer are not supported by the solver.

If you expected the solver to support your problem, you may have an error in your formulation. Otherwise, consider using a different solver.

The list of available solvers, along with the problem types they support, is available at https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers.

In [196]:
using JuMP, Pajarito, HiGHS, Hypatia

# setup solvers
oa_solver = optimizer_with_attributes(HiGHS.Optimizer,
    MOI.Silent() => true,
    "mip_feasibility_tolerance" => 1e-8,
    "mip_rel_gap" => 1e-6,
)
conic_solver = optimizer_with_attributes(Hypatia.Optimizer, 
    MOI.Silent() => true,
)
opt = optimizer_with_attributes(Pajarito.Optimizer,
    "time_limit" => 60, 
    "oa_solver" => oa_solver, 
    "conic_solver" => conic_solver,
)

n_degs = 2
n_stints = 3
n_laps = 70

# setup model
model = Model(opt)
@variable(model, x[1:n_stints] >= 0, Int)
@constraint(model, x .>= 2)
@constraint(model, sum(x) == n_laps)

    # M     M       S 
V = [
    0.3592  0.3592  0.629;    # k_0
    0.0752  0.0752  0.07;     # k_1_lin
]


@variable(model, hypo)
@objective(model, Min, hypo)

Q = V * diagm(x) * V'

aff = vcat(hypo, [Q[i, j] for i in 1:n_degs for j in 1:i]...)
@constraint(model, aff in MOI.RootDetConeTriangle(n_degs))

# solve and query solution
optimize!(model)
termination_status(model)
objective_value(model)
value.(x)




#@objective(model, Min, x + 2y)
#@constraint(model, [z, x, y] in SecondOrderCone())

# solve
#optimize!(model)
#@show termination_status(model) # MOI.OPTIMAL
#@show primal_status(model) # MOI.FEASIBLE_POINT
#@show objective_value(model) # -1 - 2 * sqrt(3)
#@show value(x) # -1
#@show value(y) # -sqrt(3)
#@show value(z) # 2

solving continuous relaxation
continuous relaxation status is ALMOST_OPTIMAL
separated 0 rays before imposing integrality
 iter     cuts          obj        bound          gap
continuous subproblem status is OPTIMAL
    1       28  -2.5839e-01  -7.7309e-01   1.9919e+00
continuous subproblem status is OPTIMAL
    2       32  -6.2039e-01  -7.7214e-01   2.4460e-01
continuous subproblem status is OPTIMAL
    3       36  -7.6753e-01  -7.7088e-01   4.3590e-03
    4       38  -7.6753e-01  -7.6237e-01  -6.7252e-03
objective relative gap -0.006725168329654752 reached; terminating
OA solver finished with status OPTIMAL, after 0.10770201683044434 seconds and 38 cuts
iterative method used 4 iterations



┌ Warning: integral solution repeated
└ @ Pajarito ~/.julia/packages/Pajarito/sNYdn/src/algorithms.jl:103


3-element Vector{Float64}:
 38.0
  2.0
 30.0

model = SOSModel(solver=PajaritoSolver())

In [199]:
using Pajarito 

model = SOSModel(solver=Pajarito.PajaritoSolver())

LoadError: UndefVarError: PajaritoSolver not defined

In [167]:
[
    0.3592  1.8991;
    0.7552  0.0351;
]

2×2 Matrix{Float64}:
 0.3592  1.8991
 0.7552  0.0351

In [161]:
[
    0.3592  1.8991;
    0.7552  0.0351;
]

2×3 Matrix{Float64}:
 0.355255  0.65295   0.863435
 0.942073  0.774755  0.620855

In [166]:
x = [1 2 3]
V * diagm(x).T * V'

LoadError: MethodError: no method matching diagm(::Matrix{Int64})

You might have used a 2d row vector where a 1d column vector was required.
Note the difference between 1d column vector [1,2,3] and 2d row vector [1 2 3].
You can convert to a column vector with the vec() function.
[0mClosest candidates are:
[0m  diagm(::Any, [91m::Any[39m, [91m::Pair{<:Integer, <:ChainRulesCore.AbstractThunk}[39m, [91m::Pair{<:Integer, <:ChainRulesCore.AbstractThunk}...[39m) at ~/.julia/packages/ChainRulesCore/C73ay/src/tangent_types/thunks.jl:77
[0m  diagm([91m::Integer[39m, [91m::Integer[39m, [91m::Pair{<:Integer, <:AbstractVector}...[39m) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/dense.jl:292
[0m  diagm([91m::Integer[39m, [91m::Integer[39m, [91m::AbstractVector[39m) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/LinearAlgebra/src/dense.jl:342
[0m  ...

In [25]:
using LinearAlgebra
using JuMP
using Hypatia

#V = randn(q, p)
V = [
    0.3592  0.3592  0.629;    # k_0
    0.0752  0.0752  0.07;     # k_1_lin
]


V = [
    0.6290  0.3592  0.3592;
    0.07    0.0752  0.0752;
]

q = 2
p = 3
n = 70
n_max = 10000

opt = Hypatia.Optimizer(verbose = false)
model = Model(() -> opt)

# model = JuMP.Model()
JuMP.@variable(model, np[1:p])
JuMP.@constraint(model, sum(Int.(np)) == n)
mid = n_max / 2
JuMP.@constraint(model, vcat(mid, np .- mid) in MOI.NormInfinityCone(p + 1))
JuMP.@variable(model, hypo)
JuMP.@objective(model, Max, hypo)

# hypogeomean + epinormeucl formulation
JuMP.@variable(model, L[i in 1:q, j in 1:i])
JuMP.@variable(model, W[1:p, 1:q])
VW = V * W
JuMP.@constraints(
    model,
    begin
        [i in 1:q, j in 1:i], VW[i, j] == L[i, j]
        [i in 1:q, j in (i + 1):q], VW[i, j] == 0
        vcat(hypo, [L[i, i] for i in 1:q]) in MOI.GeometricMeanCone(1 + q)
        [i in 1:p], vcat(sqrt(q) * np[i], W[i, :]) in JuMP.SecondOrderCone()
    end
)

# solve and query solution
optimize!(model)
termination_status(model)
objective_value(model)
value.(np)

LoadError: MethodError: no method matching Int64(::VariableRef)
[0mClosest candidates are:
[0m  (::Type{T})([91m::AbstractChar[39m) where T<:Union{Int32, Int64} at char.jl:51
[0m  (::Type{T})([91m::AbstractChar[39m) where T<:Union{AbstractChar, Number} at char.jl:50
[0m  (::Type{T})([91m::BigInt[39m) where T<:Union{Int128, Int16, Int32, Int64, Int8} at gmp.jl:359
[0m  ...

In [20]:
using LinearAlgebra
using JuMP
using Hypatia

using JuMP, Pajarito, HiGHS, Hypatia

#V = randn(q, p)
V = [
    0.3592  0.3592  0.629;    # k_0
    0.0752  0.0752  0.07;     # k_1_lin
]


V = [
    0.6290  0.3592  0.3592;
    0.07    0.0752  0.0752;
]

q = 2
p = 3
n = 70
n_max = 10000


# setup solvers
oa_solver = optimizer_with_attributes(HiGHS.Optimizer,
    MOI.Silent() => true,
    "mip_feasibility_tolerance" => 1e-8,
    "mip_rel_gap" => 1e-6,
)
conic_solver = optimizer_with_attributes(Hypatia.Optimizer, 
    MOI.Silent() => true,
)
opt = optimizer_with_attributes(Pajarito.Optimizer,
    "time_limit" => 60, 
    "oa_solver" => oa_solver, 
    "conic_solver" => conic_solver,
)

model = Model(opt)
JuMP.@variable(model, np[1:p] .>= 2, Int)
JuMP.@constraint(model, sum(np) == n)
mid = n_max / 2
JuMP.@constraint(model, vcat(mid, np .- mid) in MOI.NormInfinityCone(p + 1))
JuMP.@variable(model, hypo)
JuMP.@objective(model, Max, hypo)

# hypogeomean + epinormeucl formulation
JuMP.@variable(model, L[i in 1:q, j in 1:i])
JuMP.@variable(model, W[1:p, 1:q])
VW = V * W
JuMP.@constraints(
    model,
    begin
        [i in 1:q, j in 1:i], VW[i, j] == L[i, j]
        [i in 1:q, j in (i + 1):q], VW[i, j] == 0
        vcat(hypo, [L[i, i] for i in 1:q]) in MOI.GeometricMeanCone(1 + q)
        [i in 1:p], vcat(sqrt(q) * np[i], W[i, :]) in JuMP.SecondOrderCone()
    end
)

# solve and query solution
optimize!(model)
termination_status(model)
objective_value(model)
value.(np)

solving continuous relaxation
continuous relaxation status is OPTIMAL
separated 0 rays before imposing integrality
 iter     cuts          obj        bound          gap
continuous subproblem status is OPTIMAL
    1       33  -7.3557e+00  -7.3678e+00   1.6367e-03
separation cuts could not be added
new incumbent
    2       33  -7.3557e+00  -7.3557e+00   0.0000e+00
objective relative gap 0.0 reached; terminating
OA solver finished with status OPTIMAL, after 0.011727094650268555 seconds and 33 cuts
iterative method used 2 iterations



┌ Warning: integral solution repeated
└ @ Pajarito ~/.julia/packages/Pajarito/sNYdn/src/algorithms.jl:103


3-element Vector{Float64}:
 37.0
 31.0
  2.0